In [ ]:
from time import sleep
import requests
import pandas as pd
from tqdm import tqdm
import os
from sqlalchemy import create_engine
import psycopg2

In [ ]:
#initial API request 
#necesary to obtain object IDs which will be used in second pass to obtain image urls
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/objects')

In [ ]:
#extract object IDs as a list
indices = r.json()
ident = indices['objectIDs']

In [ ]:
#useful counter for if any issues arise for restarting from where error was thrown
#given that this is a multiday process to scrape, its less of a question of if and more a question of when
total = 0
#function to obtain image links
def get_objects_with_info(start_num):
    #results stored as dict
    #since some objects have more than 1 image, the other_links list stores any supplemental photos
    image_items= {'id': [], 'link': [], 'other_links':[]}
    #iterate through object IDs from given start which is intially zero
    for i in tqdm(ident[start_num:]):
        #API throws errors with some regularity so try/except was important
        try:
            #calls second API using specific object ID to get image urls
            o = api_call(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{i}')
            item = o.json()
            #increase total for keeping track
            total += 1
            #if item has images, appends relevant info to appropriate dictionary list
            if item['primaryImage'] != '':
                image_items['id'].append(i)
                image_items['link'].append(item['primaryImage'])
                image_items['other_links'].append(item['additionalImages'])
        except:
            #if there is an error, return dictionary so it can be saved
            #print total for reference for restarting 
            return image_items
            print(total)
            break
    return image_items


In [ ]:
#utility function to transfer dictionaries to DataFrames and then save them as csv for safe keeping
def data_saver(data_dict, csv_number):
    df = pd.DataFrame(series_1)
    df.to_csv(f'contains_images_object_ids{csv_number}.csv')
    return df

In [ ]:
#example of two functions from above working in unison to scrape and then save data
scrape_1 = get_object_with_info(0)
df_1 = data_saver(scrape_1)

In [ ]:
#once all data is scraped, it is stored in a postgres table 
#first step is to collect all the data stored in csv
data_list = [i for i in os.listdir() if i.startswith('contains_images_object_ids')]
#create DataFrame from first csv that all the others can be added to afterwards
all_data = pd.read_csv(data_list[0])
#iterate through other csvs and add to first
for csv in data_list[1:]:
    addition = pd.read_csv(csv)
    all_data = all_data.append(addition, ignore_index = True)

In [ ]:
#establish postgres connection settings
db_user = 'postgres'
db_password = ''
db_host = 'localhost'
db_port = 5432
database = 'met_data'

conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{database}'
conn = psycopg2.connect(conn_str)
conn.autocommit = True

In [ ]:
#create connection and save data as postgres table
engine = create_engine(conn_str)
all_data.to_sql('image_url', engine)